## Ollama 
* genration and chat completion
    * [Ollama  completion refrence](https://github.com/ollama/ollama/blob/main/docs/api.md#generate-a-chat-completion).

In [ ]:
import requests
import json


# Get the API URL and model from the settings
generation_api_url = "http://localhost:11434/api/generate"
model = "llama3.2:latest"
text = "Hello, how are you?"

# Send the request to the Ollama API
response = requests.post(
    generation_api_url,
    json={
        "model": model,  # Use the model variable
        "prompt": f"Summarize the following text in 3 bullet points: {text}",  # Fixed syntax
        "stream": False,
        "options": {
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
            "repetition_penalty": 1.2
        }
    }
)

In [14]:
response.json()

{'model': 'llama3.2:latest',
 'created_at': '2025-05-01T16:50:36.442025314Z',
 'response': "There is no text to summarize. The provided text only contains a greeting. If you'd like to share the actual text, I'd be happy to help!",
 'done': True,
 'done_reason': 'stop',
 'context': [128006,
  9125,
  128007,
  271,
  38766,
  1303,
  33025,
  2696,
  25,
  6790,
  220,
  2366,
  18,
  271,
  128009,
  128006,
  882,
  128007,
  271,
  9370,
  5730,
  553,
  279,
  2768,
  1495,
  304,
  220,
  18,
  17889,
  3585,
  25,
  22691,
  11,
  1268,
  527,
  499,
  30,
  128009,
  128006,
  78191,
  128007,
  271,
  3947,
  374,
  912,
  1495,
  311,
  63179,
  13,
  578,
  3984,
  1495,
  1193,
  5727,
  264,
  43213,
  13,
  1442,
  499,
  4265,
  1093,
  311,
  4430,
  279,
  5150,
  1495,
  11,
  358,
  4265,
  387,
  6380,
  311,
  1520,
  0],
 'total_duration': 1901817504,
 'load_duration': 19271281,
 'prompt_eval_count': 43,
 'prompt_eval_duration': 13908170,
 'eval_count': 33,
 'eval_d

In [15]:
response.json()['response']

"There is no text to summarize. The provided text only contains a greeting. If you'd like to share the actual text, I'd be happy to help!"

In [16]:
from pprint import pprint as pprint
pprint(response.json()['response'])

('There is no text to summarize. The provided text only contains a greeting. '
 "If you'd like to share the actual text, I'd be happy to help!")


In [ ]:
from helpers import get_settings
from openai import OpenAI
import os 

setting = get_settings()

client = OpenAI(
    api_key = setting.OPENAI_API_KEY
)

llm_model = setting.LLM_MODEL

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import torch

from langchain_huggingface import HuggingFacePipeline

In [ ]:
# Model and tokenizer setup
model_id = env_values['MODEL_ID']

# Configure 8-bit quantization
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                quantization_config=quantization_config,
                                                device_map='auto')

In [ ]:
# Set up pipeline
pipe = pipeline("text-generation",
                model=base_model,
                tokenizer=tokenizer,
                max_length=256,
                truncation=True,  # Explicitly enable truncation
                do_sample=True,
                temperature=0.6,
                top_p=0.95,
                repetition_penalty=1.2)

# Initialize LangChain HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Suggest 2 ways to lose my weight.

Answer:"""

print(llm.invoke(template))

In [ ]:
prompt_1 = """
Suggest 2 ways to lose my weight.
""".strip()

prompt_2 = """
Tell me a joke
""".strip()

llm_results = llm.generate([ prompt_1, prompt_2 ])
llm_results.generations[1][0].text

In [ ]:
# Direct pipeline usage
pipeline_output = pipe(template, return_full_text=False)
print("Pipeline Output:", pipeline_output[0]['generated_text'])